In [ ]:
import numpy as np
from sklearn.feature_selection import f_regression
from fedot.core.pipelines.node import PipelineNode
from fedot.core.pipelines.pipeline import Pipeline
import os
import json
import pandas as pd
from matplotlib import pyplot as plt

from utils import seed_everything
from utils import load_datasets
from pipeline import Experiment, ModelFactory, Evaluator
from fedot import Fedot

In [ ]:
!unzip Gd_fps.zip

In [ ]:
SEED = 42

In [ ]:
seed_everything(SEED)

In [ ]:
datasets = load_datasets("Gd_fps")

In [ ]:
lgbm_kwargs = dict(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=15,            
    min_data_in_leaf=5,      
    random_state=SEED
)

In [ ]:
from fedot.core.pipelines.pipeline_builder import PipelineBuilder

initial_assumption = PipelineBuilder().add_node('lgbmreg', params=lgbm_kwargs).build()

In [ ]:
from fedot.core.repository.metrics_repository import RegressionMetricsEnum

fedot_kwargs = dict(
    problem='regression',
    timeout=60,
    n_jobs=-1,
    logging_level=50,
    initial_assumption=initial_assumption,
    max_depth=3,
    history_dir="fedot_history",
    seed=SEED,
    preset="stable",
    metric='rmse'
)

In [ ]:
n_splits = None

In [ ]:
fedot = ModelFactory(Fedot, fedot_kwargs)

In [ ]:
evaluator = Evaluator()

In [ ]:
experiment = Experiment(
    datasets=datasets,
    target_col="lgK",
    results_dir='results',
    model_factory=fedot,
    n_splits=n_splits,
    seed=SEED,
    var_threshold=0.01,
    fs_func=None,
    k_percents=None,
    evaluator=evaluator,
)

In [ ]:
results = experiment.run()

In [13]:
with open("automl_results/automl_results.json", "w") as f:
    json.dump(results, f, indent=4)

In [14]:
def build_pipeline_summary(results_data):
    rows = []
    for dataset, dataset_data in results_data.items():
        for k_percent, info in dataset_data.items():
            fs_func = info['fs_func']
            
            # --- CV results, one row per fold ---
            cv_scores = info['cv_scores']
            if cv_scores:
                for fold_id, metrics in cv_scores.items():
                    # Parse pipeline JSON for this fold
                    pipe_path = metrics['pipeline_path']
                    if not os.path.exists(pipe_path):
                        print(f"Warning: missing pipeline file {pipe_path}")
                        continue
                    with open(pipe_path, "r") as f:
                        pipeline = json.load(f)
    
                    rows.append({
                        "dataset": dataset,
                        "features_percent": k_percent,
                        "fs_func": fs_func,
                        "pipeline_ops": pipeline.get("total_pipeline_operations", []),
                        "depth": pipeline.get("depth", None),
                        "fold": fold_id,
                        "stage": "cv",  # cross-validation
                        "rmse": metrics['rmse'],
                        "mae": metrics['mae'],
                        "r2": max(metrics['r2'], 0)
                    })
            
            # --- Train/test row ---
            test_scores = info['test_scores']
            pipe_path = test_scores['pipeline_path']
            if os.path.exists(pipe_path):
                with open(pipe_path, "r") as f:
                    pipeline = json.load(f)

                rows.append({
                    "dataset": dataset,
                    "features_percent": k_percent,
                    "fs_func": fs_func,
                    "pipeline_ops": pipeline.get("total_pipeline_operations", []),
                    "depth": pipeline.get("depth", None),
                    "fold": None,
                    "stage": "train_test",  # final evaluation
                    "rmse": test_scores["rmse"],
                    "mae": test_scores["mae"],
                    "r2": test_scores["r2"]
                })

    return pd.DataFrame(rows)

summary_df = build_pipeline_summary(results)

In [15]:
summary_df.to_csv("automl_results/pipeline_summary.csv", index=False)

In [16]:
train_test_df = summary_df[summary_df['stage'] == 'train_test']

In [17]:
with open("baseline_results.json", "r") as f:
    baseline_data = json.load(f)

In [18]:
baseline_records = []
for dataset, metrics in baseline_data.items():
    baseline_records.append({
        "dataset": dataset[:-4],  # remove .csv suffix
        "baseline_rmse": np.sqrt(metrics['holdout_metrics']['RMSE']),
        "baseline_mae": metrics['holdout_metrics']['MAE'],
        "baseline_r2": max(metrics['holdout_metrics']['R2'], 0)
    })
baseline_records = pd.DataFrame(baseline_records)

In [19]:
train_test_df = train_test_df.merge(baseline_records, on="dataset", how="left")

In [20]:
train_test_df.to_csv("automl_results/results_comparison.csv", index=False)